In [11]:
from pyChatGPT import ChatGPT
import json
from tqdm import tqdm
import pandas as pd
from collections import defaultdict
import time
from wrapt_timeout_decorator import *
from datetime import datetime
import numpy as np
from math import ceil
import pickle
import argparse
import os, re, json
from dotenv import load_dotenv

In [12]:
os.getcwd()

'/Users/UmarJahangir/Projects/ChatGPT/NaturalQuestions'

In [13]:
path = "./dataset"
os.listdir(path)

['NQ-open.train.jsonl', 'responses.json']

In [17]:
def get_start_index(responses_path):
    start_index = None
    responses = []
    with open(responses_path,'r',encoding='utf-8') as f:
            responses = json.loads(f.read())
    for i in range(len(responses)):
        if responses[i]["answer"] == "None":
            start_index = i
            break
    for index, record in enumerate(responses[start_index-1:start_index+5]):
        print(record)

    print("\nStarting from index = ", start_index, responses[start_index])
    return start_index, responses

In [18]:
responses_path = f'{path}/responses.json'
get_start_index(responses_path)

{'answer': 'Chris Long last played football in the 2019 NFL season for the Philadelphia Eagles.\n\n', 'question': 'who did chris long play football for last year'}
{'answer': 'None', 'question': 'when did harry potter and the deathly hallows come out'}
{'answer': 'None', 'question': 'how many episode of my hero academia season 2'}
{'answer': 'None', 'question': 'during the rebuttal section of a formal debate which team speaks first'}
{'answer': 'None', 'question': 'when was the last amendment made to the us constitution'}
{'answer': 'None', 'question': 'starch based food to be made into plastic'}

Starting from index =  36 {'answer': 'None', 'question': 'when did harry potter and the deathly hallows come out'}


(36,
 [{'answer': 'Hot Tub Time Machine was primarily filmed in Vancouver, Canada.\n\n',
   'question': 'where did they film hot tub time machine'},
  {'answer': 'In international waters, ships are governed by the International Regulations for Preventing Collisions at Sea (COLREGs). These regulations determine the right of way between ships. However, there is no concept of right of way in international waters in the sense of a right of way as on land. The COLREGs require ships to give way to certain vessels, such as vessels engaged in fishing or vessels not under command.\n\n',
   'question': 'who has the right of way in international waters'},
  {'answer': 'Annie Leonhart works for the Military Police Brigade in the anime and manga series Attack on Titan.\n\n',
   'question': 'who does annie work for attack on titan'},
  {'answer': 'The Immigration Reform and Control Act (IRCA) was passed on November 6, 1986.\n\n',
   'question': 'when was the immigration reform and control act passed

In [20]:
@timeout(5)
def mytest(message):
    # this example does NOT work on windows, please check the section
    # "use with Windows" in the README.rst
    print(message)
    for i in range(1,10):
        time.sleep(1)
        print('{} seconds have passed'.format(i))

if __name__ == '__main__':
    try:
        mytest('starting')
    except TimeoutError:
        print("Timeout !")
    except:
        print("Random error")

starting
1 seconds have passed
2 seconds have passed
3 seconds have passed
4 seconds have passed
Timeout !


In [25]:
def session_init():
    load_dotenv()
    token = os.getenv("GPT_TOKEN")
    api = ChatGPT(token)
    return api

In [60]:
@timeout(15)
def get_response(index, record, api):
    query = record["question"]
    response = api.send_message(query)
    answer = response["message"]
    print("=" * 20)
    print(index, answer)
    print("=" * 20)

    elif answer != "":
        return answer  

In [67]:
def get_all_responses(start_index, responses, api):
    for index, record in enumerate(responses[start_index:]):
        try:
            print(index, record)
            answer = get_response(index, record, api)

            if answer != "":
                record["answer"] = answer
                with open(responses_path, 'w') as f:
                    json.dump(responses, f)
                time.sleep(0.5)
                start_index += 1
                print(start_index)

            else:
                print(f"Stopped at: {index}")

        except ValueError as ve:
            print(str(ve))
            time.sleep(60)
            
            
    with open(responses_path, 'w') as f:
        json.dump(responses, f)

In [68]:
start_index, responses = get_start_index(responses_path)

{'answer': 'The "Big Three" triplets on the television show "This Is Us" were born on February 28, 1980.\n\n', 'question': 'when were the triplets born on this is us'}
{'answer': 'None', 'question': 'what degree do you get out of medical school'}
{'answer': 'None', 'question': "what was the source of alpha particles in rutherford's experiment"}
{'answer': 'None', 'question': 'who plays glenn on the good doctor episode 7'}
{'answer': 'None', 'question': 'who does patrick stewart play in star trek'}
{'answer': 'None', 'question': 'real name of ginny weasley in harry potter'}

Starting from index =  62 {'answer': 'None', 'question': 'what degree do you get out of medical school'}


In [69]:
def get_responses_pipeline(path):
    responses_path = f'{path}/responses.json'
    start_index, responses = get_start_index(responses_path)
    while start_index != start_index+5:
        try:
            api = session_init()
            get_all_responses(start_index, responses, api)

        except TimeoutError:
            start_index, responses = get_start_index(responses_path)
            print("Timed out! on ", start_index, responses[start_index])
            api.refresh_chat_page()
            api.reset_conversation()
            print("Sleeping for 2 seconds before trying again!")
            time.sleep(2)
            get_all_responses(start_index, responses, api)

In [70]:
get_responses_pipeline(path)

{'answer': 'The "Big Three" triplets on the television show "This Is Us" were born on February 28, 1980.\n\n', 'question': 'when were the triplets born on this is us'}
{'answer': 'None', 'question': 'what degree do you get out of medical school'}
{'answer': 'None', 'question': "what was the source of alpha particles in rutherford's experiment"}
{'answer': 'None', 'question': 'who plays glenn on the good doctor episode 7'}
{'answer': 'None', 'question': 'who does patrick stewart play in star trek'}
{'answer': 'None', 'question': 'real name of ginny weasley in harry potter'}

Starting from index =  62 {'answer': 'None', 'question': 'what degree do you get out of medical school'}
0 {'answer': 'None', 'question': 'what degree do you get out of medical school'}
Too many requests in 1 hour. Try again later.
1 {'answer': 'None', 'question': "what was the source of alpha particles in rutherford's experiment"}


NameError: name 'ReadTimeoutError' is not defined

In [103]:
with open(responses_path, 'w') as f:
                json.dump(responses, f)

In [18]:
output_arr = []
for index, query in enumerate(data_col[:5]):
    try:
        response = api.send_message(query)
        print("=" * 20)
        print(response)
        print("=" * 20)

        if response["message"] != "":
            fields = [index, query, response["message"]]
            output_arr.append(fields)
        else:
            print(f"Stopped at: {index}")

#         with open("./start", "w") as f:
#             f.write(str(index))
#         f.close()
    except ValueError as ve:
        print(ve)
        if str(ve) == "Too many requests in 1 hour. Try again later.":
            print(str(ve))
        if (
            str(ve)
            == "Only one message at a time. Please allow any other responses to complete before sending another message, or wait one minute."
        ):
            print(str(ve))
            time.sleep(60)

{'message': 'Hot Tub Time Machine was filmed primarily in and around the city of Detroit, Michigan, USA. Some additional filming also took place in Park City, Utah.\n\n', 'conversation_id': 'e4d6bc5e-155e-40ee-9910-0f17ef1609ef'}
{'message': "In international waters, ships are governed by a set of rules called the International Regulations for Preventing Collisions at Sea (COLREGS). These regulations outline the rules of the road for ships and specify which vessels have the right of way in different situations.\n\nIn general, larger vessels must give way to smaller vessels, and vessels that are overtaking must give way to those being overtaken. Vessels that are engaged in fishing or other activities that restrict their ability to maneuver must give way to all other vessels. Additionally, vessels that are under power must give way to vessels that are under sail.\n\nIt's important to note that these rules are not absolute, and the actions of a vessel's crew can also play a role in determ

TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=109.0.5414.87)
Stacktrace:
0   6b2bc0cb62e5a283_chromedriver       0x0000000102b34fa8 6b2bc0cb62e5a283_chromedriver + 4886440
1   6b2bc0cb62e5a283_chromedriver       0x0000000102ab2643 6b2bc0cb62e5a283_chromedriver + 4351555
2   6b2bc0cb62e5a283_chromedriver       0x0000000102700b27 6b2bc0cb62e5a283_chromedriver + 477991
3   6b2bc0cb62e5a283_chromedriver       0x00000001026e924d 6b2bc0cb62e5a283_chromedriver + 381517
4   6b2bc0cb62e5a283_chromedriver       0x00000001026e8de0 6b2bc0cb62e5a283_chromedriver + 380384
5   6b2bc0cb62e5a283_chromedriver       0x00000001026e7532 6b2bc0cb62e5a283_chromedriver + 374066
6   6b2bc0cb62e5a283_chromedriver       0x00000001026e7a58 6b2bc0cb62e5a283_chromedriver + 375384
7   6b2bc0cb62e5a283_chromedriver       0x00000001026f684c 6b2bc0cb62e5a283_chromedriver + 436300
8   6b2bc0cb62e5a283_chromedriver       0x00000001026f7b52 6b2bc0cb62e5a283_

In [ ]:
output_arr

In [ ]:

with open(f"./output_raw/DialFact_{int(start)}.csv", "a") as fp:
    writer = csv.writer(fp)
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        query = row["query"]

        print("*" * 20)
        print(query)
        print("*" * 20)

        try:
            response = api.send_message(query)
            print("=" * 20)
            print(response)
            print("=" * 20)

            if response["message"] != "":
                fields = [index, query, response["message"]]
                writer.writerow(fields)
            else:
                print(f"Stopped at: {index}")

            with open("./start", "w") as f:
                f.write(str(index))
            f.close()
        except ValueError as ve:
            print(ve)
            if str(ve) == "Too many requests in 1 hour. Try again later.":
                print(str(ve))
                f = open("change_token", "w")
                f.write("True")
                f.close()
            if (
                str(ve)
                == "Only one message at a time. Please allow any other responses to complete before sending another message, or wait one minute."
            ):
                print(str(ve))
                time.sleep(60)

fp.close()

api.reset_conversation()  # reset the conversation
api.clear_conversations()  # clear all conversations
api.refresh_chat_page()  # refresh the chat page


In [ ]:
result_list = []
for i in range(df.shape[0]):
    result_list.append({
        "input": input[i],
        "output": output[i]
    })

out = defaultdict(lambda: defaultdict(lambda: {}))
out = {
    "author": "IIT",
    "dataset": DATASET,
    "part": part,
    "created": TIMESTAMP,
    "result": result_list
}

with open(f"./processed/DialFact_{part}.json", "w") as fp:
    json.dump(out, fp)